## Importing Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Logistic Regression 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.utils import resample


# Random Forest 
from sklearn.preprocessing import StandardScaler

#XGBoost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

## Reading in the Data

In [2]:
df = pd.read_csv('data/filtered_data.csv')
# df = pd.read_csv('data/filtered_data_wider.csv')

In [3]:
df

,Unnamed: 0,gameid,GameDate,ab,pitchnum,inning,teambat,balls,strikes,outs,...,inducedvertbreak,platelocside,platelocheight,hometeam_id,Home,awayteam_id,Visitor,venue_id,venue_name,strikeout_binary
0,8,2021/04/17/nynmlb-colmlb-2,4/16/2021 20:33,32,4,4.0,1,1.0,2.0,1.0,...,5.260902,-0.114617,2.546066,115,Colorado Rockies,121,New York Mets,19,Coors Field,1
1,11,2021/04/17/nynmlb-colmlb-2,4/16/2021 20:33,33,3,4.0,1,0.0,2.0,2.0,...,6.102530,0.880186,1.700091,115,Colorado Rockies,121,New York Mets,19,Coors Field,0
2,30,2021/04/21/pitmlb-detmlb-2,4/21/2021 17:40,45,4,6.0,1,1.0,2.0,0.0,...,-3.835304,-0.239593,2.348037,116,Detroit Tigers,134,Pittsburgh Pirates,2394,Comerica Park,0
3,33,2021/04/25/arimlb-atlmlb-1,4/25/2021 12:20,12,4,2.0,0,1.0,2.0,1.0,...,2.129060,1.028702,2.689594,144,Atlanta Braves,109,Arizona Diamondbacks,4705,Truist Park,0
4,38,2021/04/25/arimlb-atlmlb-1,4/25/2021 12:20,12,3,2.0,0,0.0,2.0,1.0,...,13.794325,1.286732,2.807080,144,Atlanta Braves,109,Arizona Diamondbacks,4705,Truist Park,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187400,717544,2023/08/30/clemlb-minmlb-1,8/30/2023 17:10,42,5,6.0,1,1.0,2.0,0.0,...,-3.927244,-0.362439,1.593713,142,Minnesota Twins,114,Cleveland Guardians,3312,Target Field,0
187401,717550,2023/08/30/atlmlb-colmlb-1,8/31/2023 0:40,37,3,5.0,1,0.0,2.0,0.0,...,4.477882,0.436818,1.588252,115,Colorado Rockies,144,Atlanta Braves,19,Coors Field,0
187402,717573,2023/08/30/arimlb-lanmlb-1,8/31/2023 2:10,34,5,4.0,1,1.0,2.0,0.0,...,14.275191,0.586442,1.884596,119,Los Angeles Dodgers,109,Arizona Diamondbacks,22,Dodger Stadium,0
187403,717576,2023/09/02/pitmlb-slnmlb-1,9/2/2023 23:15,54,4,7.0,0,1.0,2.0,0.0,...,14.733393,-0.807034,4.470551,138,St. Louis Cardinals,134,Pittsburgh Pirates,2889,Busch Stadium,0


## Filtering the DataFrame

In [4]:
data = df[["pitcher","pitchname", "pitchresult", "eventtype","spinrate", "relspeed", "horzbreak", "inducedvertbreak", "platelocside", "platelocheight", "strikeout_binary"]].copy()

In [5]:
data = data[data['pitchname'] == 'FF'].copy()

In [ ]:
data['eventtype'].unique()

In [ ]:
data = data[~data['eventtype'].isin(['field_out'])]
data

# IF YOU UNCOMMENT, ONLY STRIKEOUTS AND BALLS

In [ ]:
# Filter the DataFrame to only include the specified event types
# data = data[data['eventtype'].isin(['strikeout', 'strikeout_double_play', 'ball', 'passed ball'])]
# # Define the list of events to remove
# irrelevant_events = [
#     'field_out', 'grounded_into_double_play', 'double_play', 'triple_play',
#     'field_error', 'defensive_indiff', 'passed_ball', 'wild_pitch', 'catcher_interf'
# ]
# 
# # Filter out rows where 'eventtype' matches any of the specified events or contains specified patterns
# data = data[
#     ~data['eventtype'].isin(irrelevant_events) & 
#     ~data['eventtype'].str.contains('caught_stealing') & 
#     ~data['eventtype'].str.contains('stolen_base') & 
#     ~data['eventtype'].str.contains('pickoff')
# ]
# 
# data = data[~data['eventtype'].str.contains('pickoff')]
# 
# # Display the filtered DataFrame
# data

In [ ]:
data['eventtype'].value_counts()

In [ ]:
data['eventtype'].unique()

### Looking at outliers

In [ ]:
numerical_features = data.select_dtypes(include=[np.number]).columns
numerical_data = data[numerical_features]

# Plot boxplots for numerical features to visualize outliers
plt.figure(figsize=(15, 10))
numerical_data.boxplot(rot=90)
plt.title('Boxplot of Numerical Features to Identify Outliers')
plt.show()

In [ ]:
# Select numerical features
numerical_features = data.select_dtypes(include=[np.number]).columns
numerical_data = data[numerical_features]
numerical_data = data[[col for col in numerical_features if col != 'spinrate']]

# Plot boxplots for numerical features to visualize outliers
plt.figure(figsize=(15, 10))
numerical_data.boxplot(rot=90)
plt.title('Boxplot of Numerical Features to Identify Outliers')
plt.show()

In [ ]:
# numerical_data = data[numerical_features]
# 
# # Function to remove outliers using the IQR method
# def remove_outliers_iqr(data, columns, threshold=100):
#     cleaned_data = data.copy()
#     for column in columns:
#         Q1 = cleaned_data[column].quantile(0.25)
#         Q3 = cleaned_data[column].quantile(0.75)
#         IQR = Q3 - Q1
#         lower_bound = Q1 - threshold * IQR
#         upper_bound = Q3 + threshold * IQR
#         # Remove rows with outliers
#         cleaned_data = cleaned_data[(cleaned_data[column] >= lower_bound) & (cleaned_data[column] <= upper_bound)]
#     return cleaned_data
# 
# # Remove outliers from the numerical features
# cleaned_data = remove_outliers_iqr(numerical_data, numerical_features)
# 
# # Display the shape of the dataset before and after removing outliers
# original_shape = numerical_data.shape
# cleaned_shape = cleaned_data.shape
# 
# original_shape, cleaned_shape

In [ ]:
# Function to remove outliers using the IQR method for a given dataset
def remove_outliers_iqr_classwise(data, columns, threshold=1.5):
    cleaned_data = data.copy()
    for column in columns:
        Q1 = cleaned_data[column].quantile(0.25)
        Q3 = cleaned_data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        # Remove rows with outliers
        cleaned_data = cleaned_data[(cleaned_data[column] >= lower_bound) & (cleaned_data[column] <= upper_bound)]
    return cleaned_data

# Separate the data into classes
strikeout_data = data[data['strikeout_binary'] == 1]
non_strikeout_data = data[data['strikeout_binary'] == 0]

# Remove outliers separately for each class
strikeout_data_cleaned = remove_outliers_iqr_classwise(strikeout_data, numerical_features)
non_strikeout_data_cleaned = remove_outliers_iqr_classwise(non_strikeout_data, numerical_features)

# Concatenate the cleaned data from both classes
cleaned_data_classwise = pd.concat([strikeout_data_cleaned, non_strikeout_data_cleaned])

# Check the distribution of the target variable in the cleaned dataset
strikeout_distribution_classwise = cleaned_data_classwise['strikeout_binary'].value_counts()
strikeout_distribution_classwise


### Looking @ histograms of features in nonstrike/strike scenarios:
- I don't think spinrate, release speed, or horz break on their own have meaningful differences
- I'm going to try combining vertical break & release speed 
- I think platelocside and platelocheight are going to be the most important 


In [ ]:
# Plot histograms of numerical features before and after outlier removal for non-strikeouts and strikeouts

# Original strikeout and non-strikeout data
original_strikeout_data = data[data['strikeout_binary'] == 1]
original_non_strikeout_data = data[data['strikeout_binary'] == 0]

# Features to visualize
features_to_visualize = ['spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak', 'platelocside', 'platelocheight']

# Plot before and after for each feature
fig, axes = plt.subplots(len(features_to_visualize), 2, figsize=(15, 25))
fig.suptitle('Comparison of Feature Distributions Before and After Outlier Removal', fontsize=16, y=1.02)

for i, feature in enumerate(features_to_visualize):
    # Original data (non-strikeouts)
    sns.histplot(original_non_strikeout_data[feature], bins=30, kde=True, color='blue', ax=axes[i, 0])
    axes[i, 0].set_title(f'{feature} - Non-strikeouts (Original)')
    axes[i, 0].set_xlabel('')

    # Cleaned data (non-strikeouts)
    sns.histplot(non_strikeout_data_cleaned[feature], bins=30, kde=True, color='green', ax=axes[i, 1])
    axes[i, 1].set_title(f'{feature} - Non-strikeouts (Cleaned)')
    axes[i, 1].set_xlabel('')

plt.tight_layout()
plt.show()

# Plot for strikeouts
fig, axes = plt.subplots(len(features_to_visualize), 2, figsize=(15, 25))
fig.suptitle('Comparison of Feature Distributions Before and After Outlier Removal (Strikeouts)', fontsize=16, y=1.02)

for i, feature in enumerate(features_to_visualize):
    # Original data (strikeouts)
    sns.histplot(original_strikeout_data[feature], bins=30, kde=True, color='red', ax=axes[i, 0])
    axes[i, 0].set_title(f'{feature} - Strikeouts (Original)')
    axes[i, 0].set_xlabel('')

    # Cleaned data (strikeouts)
    sns.histplot(strikeout_data_cleaned[feature], bins=30, kde=True, color='orange', ax=axes[i, 1])
    axes[i, 1].set_title(f'{feature} - Strikeouts (Cleaned)')
    axes[i, 1].set_xlabel('')

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set Seaborn style for consistency and aesthetics
sns.set(style="whitegrid")

# Define a color palette for strikeouts and non-strikeouts
colors = {"non_strikeouts": "#4C72B0", "strikeouts": "#C44E52"}

# Plot cleaned data for each feature (horizontal layout with larger plots and enhancements)
fig, axes = plt.subplots(2, len(features_to_visualize), figsize=(30, 15))
fig.suptitle('Comparison of Feature Distributions for Cleaned Data: Strikeouts vs. Non-Strikeouts', fontsize=24, y=1.02, weight='bold', color='#333333')

for i, feature in enumerate(features_to_visualize):
    # Determine the x-axis limits based on combined data for consistency
    x_min = min(non_strikeout_data_cleaned[feature].min(), strikeout_data_cleaned[feature].min())
    x_max = max(non_strikeout_data_cleaned[feature].max(), strikeout_data_cleaned[feature].max())

    # Cleaned data (non-strikeouts)
    sns.histplot(non_strikeout_data_cleaned[feature], bins=30, kde=True, color=colors["non_strikeouts"], 
                 ax=axes[0, i], linewidth=1.5, alpha=0.8)
    axes[0, i].set_title(f'Non-strikeouts ({feature})', fontsize=16, color='#4C72B0', weight='bold')
    axes[0, i].set_xlabel('')
    axes[0, i].set_ylabel('Count', fontsize=14)
    axes[0, i].set_xlim(x_min, x_max)
    axes[0, i].tick_params(axis='both', labelsize=12)
    axes[0, i].grid(True, linestyle='--', linewidth=0.7, alpha=0.5)

    # Cleaned data (strikeouts)
    sns.histplot(strikeout_data_cleaned[feature], bins=30, kde=True, color=colors["strikeouts"], 
                 ax=axes[1, i], linewidth=1.5, alpha=0.8)
    axes[1, i].set_title(f'Strikeouts ({feature})', fontsize=16, color='#C44E52', weight='bold')
    axes[1, i].set_xlabel('')
    axes[1, i].set_ylabel('Count', fontsize=14)
    axes[1, i].set_xlim(x_min, x_max)
    axes[1, i].tick_params(axis='both', labelsize=12)
    axes[1, i].grid(True, linestyle='--', linewidth=0.7, alpha=0.5)

# Adjust the layout to give more space between plots
plt.tight_layout(pad=5.0, w_pad=4.5, h_pad=4.0)
plt.subplots_adjust(top=0.92)
plt.show()


## Plotly Plot!

In [ ]:
# Create a new variable that is relspeed and inducedvertbreak multiplied together
cleaned_data_classwise['relspeed_inducedvertbreak'] = cleaned_data_classwise['relspeed'] * cleaned_data_classwise['inducedvertbreak']

# Separate data into strikeouts and non-strikeouts
strikeout_data = cleaned_data_classwise[cleaned_data_classwise['strikeout_binary'] == 1]
non_strikeout_data = cleaned_data_classwise[cleaned_data_classwise['strikeout_binary'] == 0]

# Plot the distribution of the new variable for strikeouts and non-strikeouts
plt.figure(figsize=(12, 6))
sns.histplot(strikeout_data['relspeed_inducedvertbreak'], bins=30, kde=True, color='red', label='Strikeouts')
sns.histplot(non_strikeout_data['relspeed_inducedvertbreak'], bins=30, kde=True, color='green', label='Non-strikeouts')
plt.title('Distribution of relspeed * inducedvertbreak for Strikeouts vs. Non-strikeouts')
plt.xlabel('relspeed * inducedvertbreak')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
non_strikeout_data['eventtype'].unique()

In [ ]:
feature_columns = [
        'spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak', 
        'platelocside', 'platelocheight', 'relspeed_inducedvertbreak'
    ]
cleaned_data_classwise[feature_columns].corr()

### If I want to keep just the interaction term:

In [ ]:
# Count the appearances for each pitcher and filter out those with fewer than 50 appearances
pitcher_appearances = cleaned_data_classwise['pitcher'].value_counts()
pitchers_with_100_plus_appearances = pitcher_appearances[pitcher_appearances >= 100].index

# Filter the dataframe to include only pitchers with 50 or more appearances
cleaned_data_classwise = cleaned_data_classwise[cleaned_data_classwise['pitcher'].isin(pitchers_with_100_plus_appearances)]

cleaned_data_classwise

## Ranking

In [ ]:
# Calculate the number of 'strikeout' and 'ball' events for each pitcher
strike_counts = cleaned_data_classwise[cleaned_data_classwise['eventtype'] == 'strikeout'].groupby('pitcher').size()
ball_counts = cleaned_data_classwise[cleaned_data_classwise['eventtype'] == 'ball'].groupby('pitcher').size()

# Combine these counts into a single DataFrame
pitcher_stats = pd.DataFrame({'strikeouts': strike_counts, 'balls': ball_counts}).fillna(0)

pitcher_stats = pitcher_stats[pitcher_stats['balls'] > 0]

# Calculate the strike-to-ball ratio
pitcher_stats['strike_to_ball_ratio'] = pitcher_stats['strikeouts'] / pitcher_stats['balls']

# Get the top 10 pitchers by strike-to-ball ratio
top_10_pitchers = pitcher_stats.sort_values(by='strike_to_ball_ratio', ascending=False).head(10)

# Display the result
top_10_pitchers

In [ ]:
# Get the top 10 pitchers by strike-to-ball ratio
bottom_10_pitchers = pitcher_stats.sort_values(by='strike_to_ball_ratio', ascending=True).head(10)

# Display the result
bottom_10_pitchers

In [ ]:
# # Add the new variable to the DataFrame
# # cleaned_data_classwise['VelocityBreakProduct'] = cleaned_data_classwise['relspeed'] * cleaned_data_classwise['inducedvertbreak']
# 
# # Remove the original 'relspeed' and 'inducedvertbreak' columns
# cleaned_data_classwise = cleaned_data_classwise.drop(columns=['relspeed', 'inducedvertbreak'])
# 
# # Display the first few rows of the updated DataFrame to confirm the changes
cleaned_data_classwise.head()
cleaned_data_classwise.to_csv('baseball_stuff.csv', index=False)

### More feature engineering: Making a new col 
- this col averages each pitcher's average velocity and takes into acc on a pitch by pitch basis, how much the velo differs from the pitcher's regular speed

In [ ]:
# Calculate the pitcher's average release speed
cleaned_data_classwise['average_relspeed'] = cleaned_data_classwise.groupby('pitcher')['relspeed'].transform('mean')

# Calculate the difference between each pitch's release speed and the pitcher's average release speed
cleaned_data_classwise['relspeed_diff'] = cleaned_data_classwise['relspeed'] - cleaned_data_classwise['average_relspeed']

In [ ]:
# Separate data into strikeouts and non-strikeouts
strikeout_data = cleaned_data_classwise[cleaned_data_classwise['strikeout_binary'] == 1]
non_strikeout_data = cleaned_data_classwise[cleaned_data_classwise['strikeout_binary'] == 0]
# Plot the distribution of the new variable for strikeouts and non-strikeouts
plt.figure(figsize=(12, 6))
sns.histplot(strikeout_data['relspeed'], bins=30, kde=True, color='red', label='Strikeouts')
sns.histplot(non_strikeout_data['relspeed'], bins=30, kde=True, color='green', label='Non-strikeouts')
plt.title('Distribution of Release Speed Difference for Strikeouts vs. Non-strikeouts')
plt.xlabel('Release Speed Difference (relspeed - average_relspeed)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
cleaned_data_classwise

## Logistic Regression

In [ ]:
# Simplify pitchresult into broader categories
def categorize_pitchresult(pitchresult):
    if pitchresult in ['S', 'C']:  # Likely strike-related
        return 'strike_related'
    elif pitchresult in ['X', 'D', 'T', 'O']:  # In-play outcomes
        return 'in_play'
    elif pitchresult in ['B', 'W']:  # Balls and walks
        return 'ball_related'
    elif pitchresult == 'F':  # Foul
        return 'foul'
    else:  # Other miscellaneous outcomes
        return 'other'

# Apply categorization
cleaned_data_classwise['pitchresult_category'] = cleaned_data_classwise['pitchresult'].apply(categorize_pitchresult)

# One-hot encode the new categorical columns for logistic regression
data_encoded = pd.get_dummies(cleaned_data_classwise, columns=['pitchresult_category'], drop_first=True)

# Define the new feature set including the encoded pitchresult categories and original features
feature_columns = [
        'spinrate', 'average_relspeed', 'relspeed_diff', 'horzbreak', 'inducedvertbreak', 
        'platelocside', 'platelocheight', 'relspeed_inducedvertbreak'
    ] # + [col for col in data_encoded.columns if 'pitchresult_category_' in col]

# Separate features and target
X = data_encoded[feature_columns]
y = data_encoded['strikeout_binary']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train logistic regression model with class weights to handle imbalance
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

accuracy, precision, recall, roc_auc

### OverSampling

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Separate features and target
X = cleaned_data_classwise[[
        'spinrate', 'average_relspeed', 'relspeed_diff', 'horzbreak', 'inducedvertbreak', 
        'platelocside', 'platelocheight', 'relspeed_inducedvertbreak'
    ]]
y = cleaned_data_classwise['strikeout_binary']  # assuming you have a column that indicates if the outcome was a strikeout (0 or 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Apply SMOTE to oversample the minority class in the training set
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Train logistic regression model with class weights to handle imbalance
log_reg = LogisticRegression(max_iter=1000,  class_weight={0: 1, 1: 0.94})
# log_reg = LogisticRegression(max_iter=1000,  class_weight="balanced")

log_reg.fit(X_resampled, y_resampled)

# Make predictions
y_pred = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

accuracy, precision, recall, roc_auc

In [ ]:
# X_train

### Looking at the Coefficients

In [ ]:
# Get the feature names and coefficients
feature_names = X_train.columns
coefficients = log_reg.coef_[0]

# Create a DataFrame for easier interpretation
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', key=abs, ascending=False)

# Plot the coefficients for visual clarity
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'])
plt.xlabel('Coefficient Value')
plt.title('Feature Importance: Logistic Regression Coefficients')
plt.gca().invert_yaxis()
plt.show()

feature_importance

## Random Forest with NOTHING done

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the Random Forest model
accuracy_rf_og = accuracy_score(y_test, y_pred_rf)
precision_rf_og = precision_score(y_test, y_pred_rf)
recall_rf_og = recall_score(y_test, y_pred_rf)
roc_auc_rf_og = roc_auc_score(y_test, y_pred_rf_proba)

accuracy_rf_og, precision_rf_og, recall_rf_og, roc_auc_rf_og

In [ ]:
metrics_rf_og = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'ROC AUC'],
    'Value': [accuracy_rf_og, precision_rf_og, recall_rf_og, roc_auc_rf_og]
}

# Convert dictionary to a DataFrame
accuracy_metrics_rf_df_og  = pd.DataFrame(metrics_rf_og)

accuracy_metrics_rf_df_og

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model with class weights to handle imbalance
rf_model = RandomForestClassifier(n_estimators=100, class_weight={0: 1, 1: 2}, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf_proba)

accuracy_rf, precision_rf, recall_rf, roc_auc_rf

In [ ]:
metrics_rf_1 = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'ROC AUC'],
    'Value': [accuracy_rf, precision_rf, recall_rf, roc_auc_rf]
}

# Convert dictionary to a DataFrame
accuracy_metrics_rf_df_1  = pd.DataFrame(metrics_rf_1)

accuracy_metrics_rf_df_1

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import SMOTE
# 
# # Load the data
# # df = pd.read_csv(file_path)
# 
# # Simplify pitchresult into broader categories
# df['pitchresult_category'] = df['pitchresult'].apply(categorize_pitchresult)
# 
# # One-hot encode the new categorical columns for logistic regression
# data_encoded = pd.get_dummies(df, columns=['pitchresult_category'], drop_first=True)
# 
# # Define the new feature set including the encoded pitchresult categories and original features
# feature_columns = [
#     'spinrate', 'relspeed', 'horzbreak', 'inducedvertbreak', 
#     'platelocside', 'platelocheight'
# ] + [col for col in data_encoded.columns if 'pitchresult_category_' in col]
# 
# # Separate features and target
# X = data_encoded[feature_columns]
# y = data_encoded['strikeout_binary']
# 
# # Oversample the minority class using SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X, y)
# 
# # Feature scaling
# scaler = StandardScaler()
# X_resampled_scaled = scaler.fit_transform(X_resampled)
# 
# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_resampled_scaled, y_resampled, test_size=0.3, random_state=42)
# 
# # Train Random Forest model with class weights to handle imbalance
# rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
# rf_model.fit(X_train, y_train)
# 
# # Make predictions
# y_pred_rf = rf_model.predict(X_test)
# y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]
# 
# # Evaluate the Random Forest model
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
# recall_rf = recall_score(y_test, y_pred_rf)
# roc_auc_rf = roc_auc_score(y_test, y_pred_rf_proba)
# 
# accuracy_rf, precision_rf, recall_rf, roc_auc_rf

In [ ]:
# X_train

In [ ]:
# X_resampled

### DownSampling with RandomForest

In [ ]:
# Define the new feature set excluding the encoded pitchresult categories to avoid data leakage
feature_columns = [
        'spinrate', 'average_relspeed', 'relspeed_diff', 'horzbreak', 'inducedvertbreak', 
        'platelocside', 'platelocheight',  'relspeed_inducedvertbreak'
    ]


# Separate features and target
X = data_encoded[feature_columns]
y = data_encoded['strikeout_binary']

# Concatenate features and target for resampling
data_resampled = pd.concat([X, y], axis=1)

# Separate majority and minority classes
strikeout = data_resampled[data_resampled['strikeout_binary'] == 1]
non_strikeout = data_resampled[data_resampled['strikeout_binary'] == 0]

# Downsample majority class (non-strikeouts)
non_strikeout_downsampled = resample(non_strikeout, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(strikeout),  # match minority class
                                     random_state=42)

# Combine minority class with downsampled majority class
data_balanced = pd.concat([strikeout, non_strikeout_downsampled])

# Separate features and target after resampling
X_balanced = data_balanced.drop('strikeout_binary', axis=1)
y_balanced = data_balanced['strikeout_binary']

# Feature scaling
scaler = StandardScaler()
X_balanced_scaled = scaler.fit_transform(X_balanced)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced_scaled, y_balanced, test_size=0.3, random_state=42)

# Train Random Forest model with class weights to handle imbalance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf_proba)

accuracy_rf, precision_rf, recall_rf, roc_auc_rf

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
data_resampled

In [ ]:
# Interaction between spinrate and relspeed
data_encoded['spinrate_relspeed'] = data_encoded['spinrate'] * data_encoded['relspeed']

# Movement Ratio between horizontal and vertical break
data_encoded['movement_ratio'] = data_encoded['horzbreak'] / (data_encoded['inducedvertbreak'] + 1e-5)

# Normalize plate location to capture if the pitch is at the edge of the strike zone
data_encoded['is_edge_pitch'] = (
    (data_encoded['platelocside'].abs() > 0.7) |  # Assuming a threshold for edge pitches
    (data_encoded['platelocheight'] < 1.5) | 
    (data_encoded['platelocheight'] > 3.5)
).astype(int)

In [ ]:
data_encoded

In [ ]:
# Create new engineered features

# Interaction between spinrate and relspeed
data_encoded['spinrate_relspeed'] = data_encoded['spinrate'] * data_encoded['relspeed']

# Movement Ratio between horizontal and vertical break
data_encoded['movement_ratio'] = data_encoded['horzbreak'] / (data_encoded['inducedvertbreak'] + 1e-5)

# Normalize plate location to capture if the pitch is at the edge of the strike zone
data_encoded['is_edge_pitch'] = (
    (data_encoded['platelocside'].abs() > 0.7) |  # Assuming a threshold for edge pitches
    (data_encoded['platelocheight'] < 1.5) | 
    (data_encoded['platelocheight'] > 3.5)
).astype(int)

#Update the feature columns to include the new engineered features
feature_columns = [ 'relspeed_inducedvertbreak', 'relspeed',
        'spinrate', 'relspeed_diff', 'horzbreak', 'inducedvertbreak', 
        'platelocside', 'platelocheight'
    ]


# feature_columns = []
# feature_columns = ['platelocside', 'platelocheight', 'relspeed_diff', 'horzbreak']

# feature_columns = ["spinrate", "relspeed", "horzbreak", "inducedvertbreak", "platelocheight", "platelocside", "is_edge_pitch"]
#, 'relspeed_inducedvertbreak'] # - prob gonna go with this one

# feature_columns = ['average_relspeed','spinrate','platelocside', 'platelocheight', 'is_edge_pitch'] - might go with this one

# Separate features and target
X = data_encoded[feature_columns]
y = data_encoded['strikeout_binary']

# Concatenate features and target for resampling
data_resampled = pd.concat([X, y], axis=1)

# Separate majority and minority classes
strikeout = data_resampled[data_resampled['strikeout_binary'] == 1]
non_strikeout = data_resampled[data_resampled['strikeout_binary'] == 0]

# Downsample majority class (non-strikeouts)
non_strikeout_downsampled = resample(non_strikeout, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(strikeout),  # match minority class
                                     random_state=42)

# Combine minority class with downsampled majority class
data_balanced = pd.concat([strikeout, non_strikeout_downsampled])

# Separate features and target after resampling
X_balanced = data_balanced.drop('strikeout_binary', axis=1)
y_balanced = data_balanced['strikeout_binary']

# Feature scaling
scaler = StandardScaler()
X_balanced_scaled = scaler.fit_transform(X_balanced)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced_scaled, y_balanced, test_size=0.3, random_state=42)

# Train Random Forest model with class weights to handle imbalance
rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf_proba)

accuracy_rf, precision_rf, recall_rf, roc_auc_rf

In [ ]:
metrics_rf = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'ROC AUC'],
    'Value': [accuracy_rf, precision_rf, recall_rf, roc_auc_rf]
}

# Convert dictionary to a DataFrame
accuracy_metrics_rf_df  = pd.DataFrame(metrics_rf)

accuracy_metrics_rf_df 

In [ ]:
X_balanced

## XGBoost 

In [ ]:
# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0]
}

# Set up the GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=1
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best estimator and its parameters
best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the tuned XGBoost model
y_pred_best_xgb = best_xgb_model.predict(X_test)
y_pred_best_xgb_proba = best_xgb_model.predict_proba(X_test)[:, 1]

# Evaluate the tuned XGBoost model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
precision_best_xgb = precision_score(y_test, y_pred_best_xgb, zero_division=1)
recall_best_xgb = recall_score(y_test, y_pred_best_xgb)
roc_auc_best_xgb = roc_auc_score(y_test, y_pred_best_xgb_proba)

best_params, accuracy_best_xgb, precision_best_xgb, recall_best_xgb, roc_auc_best_xgb

In [ ]:
# import pickle
# 
# # Save the model
# with open('best_xgb_model.pkl', 'wb') as file:
#     pickle.dump(best_xgb_model, file)

In [ ]:
metrics_xgb = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'ROC AUC'],
    'Value': [accuracy_best_xgb, precision_best_xgb, recall_best_xgb, roc_auc_best_xgb]
}

# Convert dictionary to a DataFrame
accuracy_metrics_xgb_df  = pd.DataFrame(metrics_xgb)

accuracy_metrics_xgb_df 

In [ ]:
print(classification_report(y_test, y_pred_best_xgb))

In [ ]:
# Generate the confusion matrix for the tuned XGBoost model (best from RandomizedSearchCV)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_best_xgb)

conf_matrix_xgb

In [ ]:
X_train

In [ ]:
y_pred_best_xgb.shape

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Plot the feature importance
plt.figure(figsize=(10, 6))
plot_importance(best_xgb_model, importance_type='weight', ax=plt.gca())
plt.show()

In [ ]:
# Create a dictionary to map feature indices to the actual column names
feature_names = {f"f{i}": col for i, col in enumerate(X_balanced.columns)}

# Plot feature importance with renamed features
plt.figure(figsize=(10, 6))
ax = plot_importance(best_xgb_model, importance_type='gain', ax=plt.gca())

# Rename the feature labels
ax.set_yticklabels([feature_names[label.get_text()] for label in ax.get_yticklabels()])
plt.xlabel('XGBoost Feature importance')
plt.show()

In [ ]:
import numpy as np

# Get feature importances from the model
importances = best_xgb_model.get_booster().get_score(importance_type='gain')

# Map the feature indices to actual column names and importance values
feature_importances = np.array([(feature_names[feature], importance) for feature, importance in importances.items()])
feature_importances_dict = {feature_names[feature]: importance for feature, importance in importances.items()}

# Display or use the feature_importances array
feature_importances_dict

In [ ]:
import shap
import xgboost as xgb

# Create the SHAP explainer
explainer = shap.Explainer(best_xgb_model, X_balanced)

# Calculate SHAP values for the dataset
shap_values = explainer(X_balanced)

In [ ]:
shap.summary_plot(shap_values, X_balanced)

In [ ]:
# Assuming X_balanced is a DataFrame or numpy array
single_instance = X_balanced.iloc[[200]]  # Replace 0 with the index of the desired observation

# Calculate SHAP values for the single instance
shap_values_single = explainer(single_instance)

# Plot SHAP values for the single observation
shap.waterfall_plot(shap_values_single[0])

In [ ]:
# X_balanced
# X_balanced.to_csv('data/X_balanced.csv')

In [ ]:
# data_encoded[0:16198]
# data_encoded[0:16198].to_csv('data/data_balanced.csv')

## Attempting to Create a score based on Feature Importance
- Removed pitchers that only had one pitch, because they are irrelevant to our ranking

In [ ]:
og_data = cleaned_data_classwise
og_data 

In [ ]:
bruh = data_encoded.loc[:, data_encoded.columns.difference(['pitchresult_category_strike_related','pitchresult_category_foul', 'pitchresult_category_in_play', 'pitchresult_category_other', 'pitchresult_category_strikeout', 'pitchresult_category_strikeout_binary', 'movement_ratio'])]

In [ ]:
# Filter the data for the desired columns
data = bruh

# Further filtering for FF pitchname and excluding 'field_out' eventtype
data = data[(data['pitchname'] == 'FF') & (~data['eventtype'].isin(['field_out']))]

# Filter to only include scenarios where pitchresult == 'C' (strikeout)
data_strikeout = data[data['pitchresult'] == 'C']

# Define the pitch characteristics for analysis
pitch_characteristics = ["spinrate", "horzbreak", "inducedvertbreak", "platelocside", "platelocheight", 
                         "relspeed", "average_relspeed", "relspeed_diff", "relspeed_inducedvertbreak"]

pitcher_stats = data_strikeout.groupby("pitcher").filter(lambda x: len(x) > 1)

# Group by pitcher and calculate the mean and standard deviation of the pitch characteristics
pitcher_stats = pitcher_stats.groupby("pitcher")[pitch_characteristics].agg(['mean', 'std'])

In [ ]:
pitcher_stats.columns

In [ ]:
pitcher_stats.reset_index(inplace=True)

pitcher_stats

In [ ]:
# bruh = data_encoded.loc[:, data_encoded.columns.difference(['pitchresult_category_strike_related','pitchresult_category_foul', 'pitchresult_category_in_play', 'pitchresult_category_other', 'pitchresult_category_strikeout', 'pitchresult_category_strikeout_binary', 'movement_ratio'])]
columns = ['pitcher'] + [col for col in bruh.columns if col != 'pitcher']
bruh = bruh[columns]

In [ ]:
# Reorder columns to make "pitcher" the first column
bruh.drop(columns=['average_relspeed'], inplace=True)
bruh

## Creating Part of the Score for Release Speed!

In [ ]:
bruh_1 = bruh
bruh_1

In [ ]:
# Set pitcher as the index for pitcher_stats to allow easy lookup
pitcher_stats.set_index('pitcher', inplace=True)

# Define the function to calculate the new value
def calculate_z_score_adjusted(pitch, pitcher_stats, constant=5.74):
    pitcher = pitch['pitcher']
    relspeed = pitch['relspeed']
    
    # Get pitcher's mean and std for relspeed from pitcher_stats
    try:
        relspeed_mean = pitcher_stats['relspeed']['mean'][pitcher]
        relspeed_std = pitcher_stats['relspeed']['std'][pitcher]
        
        # Calculate z-score and multiply by the constant
        z_score_adjusted = ((relspeed - relspeed_mean) / relspeed_std) * constant
        return z_score_adjusted
    except KeyError:
        return np.nan

# Apply the function to the bruh dataframe
bruh_1['relspeed_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted(row, pitcher_stats), axis=1)

In [ ]:
bruh_1

In [ ]:
# # Set pitcher as the index for pitcher_stats to allow easy lookup
# pitcher_stats.set_index('pitcher', inplace=True)
# 
# # Define the function to calculate the new value
# def calculate_z_score_adjusted(pitch, pitcher_stats, constant=6.17):
#     pitcher = pitch['pitcher']
#     relspeed = pitch['relspeed']
#     
#     # Get pitcher's mean and std for relspeed from pitcher_stats
#     try:
#         relspeed_mean = pitcher_stats['relspeed']['mean'][pitcher]
#         relspeed_std = pitcher_stats['relspeed']['std'][pitcher]
#         
#         # Calculate z-score and multiply by the constant
#         z_score_adjusted = ((relspeed - relspeed_mean) / relspeed_std) * constant
#         return z_score_adjusted
#     except KeyError:
#         return np.nan
# 
# # Apply the function to the bruh dataframe
# bruh_1['relspeed_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted(row, pitcher_stats), axis=1)
bruh

In [ ]:
# Define the function to calculate z-score adjusted values for different characteristics
def calculate_z_score_adjusted_for_feature(pitch, pitcher_stats, feature, constant=6.17):
    pitcher = pitch['pitcher']
    value = pitch[feature]
    
    # Get pitcher's mean and std for the feature from pitcher_stats
    try:
        feature_mean = pitcher_stats[feature]['mean'][pitcher]
        feature_std = pitcher_stats[feature]['std'][pitcher]
        
        # Calculate z-score and multiply by the constant
        z_score_adjusted = ((value - feature_mean) / feature_std) * constant
        return z_score_adjusted
    except KeyError:
        return np.nan

# Apply these functions to the `bruh` dataframe
bruh_1['spinrate_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'spinrate', constant=3.78366), axis=1)
bruh_1['horzbreak_adjusted'] = bruh.apply(lambda row: abs(calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'horzbreak', constant =4.06374)), axis=1)
# bruh_1['inducedvertbreak_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'inducedvertbreak', constant = 3.),  axis=1)
bruh_1['platelocside_adjusted'] = bruh.apply(lambda row: -calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'platelocside', constant=29.64492), axis=1)
bruh_1['platelocheight_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'platelocheight', constant=31.36657), axis=1)
bruh_1['relspeed_inducedvertbreak'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'relspeed_inducedvertbreak', constant=6.96876), axis=1)

In [ ]:
bruh_1

In [ ]:
# Calculate "my_score" as the row sum of the specified columns
bruh_1['my_score'] = bruh_1[['spinrate_adjusted', 'horzbreak_adjusted', 'platelocside_adjusted', 'platelocheight_adjusted', 'relspeed_inducedvertbreak']].sum(axis=1)

# Calculate the average of "my_score" grouped by "pitcher"
average_score_by_pitcher = bruh_1.groupby('pitcher')['my_score'].mean().reset_index()

# Display the result
average_score_by_pitcher

In [ ]:
average_score_by_pitcher_sorted = average_score_by_pitcher.sort_values(by='my_score', ascending=False).reset_index(drop=True)

average_score_by_pitcher_sorted

In [ ]:
average_score_by_pitcher[average_score_by_pitcher['pitcher'] == "deGrom, Jacob"]

In [ ]:
bruh_1[bruh_1['pitcher'] == "Morton, Charlie"]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(average_score_by_pitcher['my_score'], bins=200, edgecolor='k', alpha=0.7)
plt.xlabel('My Score')
plt.ylabel('Frequency')
plt.title('Distribution of My Score Across Pitchers')
plt.show()

## Trying to compare my score against ERA and other relevant metrics

In [ ]:
# !pip install pybaseball

In [ ]:
from pybaseball import pitching_stats

# Retrieve pitching stats for 2021 to 2023
data = pitching_stats(2021, 2023)
# Filter for desired columns
era_data = data[['Season', 'Name', 'Team', 'ERA']]

In [ ]:
era_data

In [ ]:
# Calculating the overall average ERA for each pitcher across all seasons
overall_average_era = era_data.groupby('Name')['ERA'].mean().reset_index()

overall_average_era

In [ ]:
# Adjusting the names in average_score_by_pitcher to match the format in overall_average_era

# Function to reformat "Lastname, Firstname" to "Firstname Lastname"
def reformat_name(name):
    last, first = name.split(', ')
    return f"{first} {last}"

# Applying the reformatting function
average_score_by_pitcher['Name'] = average_score_by_pitcher['pitcher'].apply(reformat_name)

# Merging the dataframes on the reformatted name column
merged_data = overall_average_era.merge(average_score_by_pitcher, on="Name")

# Plotting the ERA vs. my_score for each pitcher
plt.figure(figsize=(10, 6))
plt.scatter(merged_data['ERA'], merged_data['my_score'], color='blue', alpha=0.7)

# Adding labels and title
plt.xlabel("Average ERA")
plt.ylabel("Average My Score")
plt.title("Comparison of Average ERA and Average My Score by Pitcher")
plt.grid(True)

# Displaying the plot
plt.show()

In [ ]:
merged_data

In [ ]:
# Calculate the Pearson correlation coefficient between ERA and my_score
correlation = merged_data['ERA'].corr(merged_data['my_score'])
print("Correlation between ERA and my_score:", correlation)

In [ ]:
# Filter data to include only rows where my_score is between -20 and 20
subset_data = merged_data[(merged_data['my_score'] >= 50) & (merged_data['my_score'] <= 90)]

# Calculate the correlation for the filtered subset
subset_correlation = subset_data['ERA'].corr(subset_data['my_score'])
print("Correlation between ERA and my_score in [-20, 20]:", subset_correlation)

# Calculate the correlation for the full dataset for comparison
full_correlation = merged_data['ERA'].corr(merged_data['my_score'])
print("Correlation between ERA and my_score for all data:", full_correlation)


In [ ]:
# Plotting the ERA vs. my_score for the filtered range
plt.figure(figsize=(10, 6))
plt.scatter(subset_data['ERA'], subset_data['my_score'], color='green', alpha=0.7)

# Adding labels and title
plt.xlabel("Average ERA")
plt.ylabel("Average My Score")
plt.title("Comparison of Average ERA and My Score by Pitcher (My Score between -20 and 20)")
plt.grid(True)

# Displaying the plot
plt.show()
